Given discretised hfev1 variable node, define the prior distribution from LMS model

In [1]:
import src.modelling_fev1.pred_fev1 as pred_fev1
import numpy as np
import src.models.helpers as mh
from scipy.stats import norm

In [2]:
# Define hfev1 variable node
HFEV1 = mh.VariableNode("HFEV1", 2, 6, 0.1)
HFEV1.bins

array([2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3. , 3.1, 3.2,
       3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4. , 4.1, 4.2, 4.3, 4.4, 4.5,
       4.6, 4.7, 4.8, 4.9, 5. , 5.1, 5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8,
       5.9])

In [46]:
def calc_hfev1_prior(hfev1_bins, height, age, sex):
    """
    Computes the prior fo hfev1 given its bins
    This model uses the inversed LMS function to compute the zscores of each bin given an array of hfev1/predictedFEV1 bin values
    """
    # Compute the predicted FEV1 for the individual
    fev1 = pred_fev1.calc_predicted_FEV1_LMS_straight(height, age, sex)
    S = fev1["S"]
    M = fev1["mean"]
    L = fev1["L"]

    # Define inverse LMS function
    def inverse_lms_pred_fev1(fev1_values, S, M, L):
        return (np.exp(L * np.log(fev1_values / M)) - 1) / (S * L)

    # Compute zscores for each bin
    zscores = inverse_lms_pred_fev1(hfev1_bins, S, M, L)

    # Get probabilities for each bin
    p = norm.pdf(zscores)
    p = p / p.sum()

    return p

In [48]:
# Validate inverse function
S = 2
L = 1
M = 4

z = 300
a = np.exp(np.log(1 + z * S * L) / L + np.log(M))
print(f"z={z}, a={a}")

z_bis = (np.exp(L * np.log(a / M)) - 1) / (S * L)
print(f"z_bis={z_bis}, a={a}")

z=1, a=12.0
z_bis=1.0000000000000002, a=12.0


In [60]:
p, z = pred_fev1.calc_proba_for_hfev1(HFEV1.bins, 172, 30, "Male")
print(f"sum of lms p = {sum(p)}, len(p) = {len(p)}")

print(len(HFEV1.bins))

p_gauss = norm(4.17, 0.45).pdf(HFEV1.bins)
# Normalise p_gauss
p_gauss = p_gauss / p_gauss.sum()
print(f"sum of p_gauss = {sum(p_gauss)}, len(p_gauss) = {len(p_gauss)}")


# Using plotly go, plot the probability distribution
import plotly.graph_objects as go

fig = go.Figure()
# Add trace with values from normal distribution centered on 4.17 and sigma 0.45
fig.add_trace(
    go.Scatter(x=HFEV1.bins, y=p_gauss, mode="lines+markers", name="N(4.17, 0.45)")
)
fig.add_trace(go.Scatter(x=HFEV1.bins, y=p, mode="lines+markers", name="LMS hfev1"))
fig.update_layout(title="Probability distribution for hfev1")
fig.show()

predicted FEV1: 4.17065185293416
sum of lms p = 1.0000000000000002, len(p) = 41
41
sum of p_gauss = 1.0000000000000002, len(p_gauss) = 41


In [57]:
# Calculate area under curve for p
print(f"Area under curve for p = {np.trapz(p, HFEV1.bins)}")

Area under curve for p = 0.09999604386169679


In [6]:
HFEV1.bins_str[0]

'[2.0; 2.1)'

In [10]:
last_elem = 6
bins_str = []
bins_arr = np.array([])
for i in range(len(HFEV1.bins)):
    low = round(HFEV1.bins[i], 2)
    up = HFEV1.b if i == (len(HFEV1.bins) - 1) else round(HFEV1.bins[i + 1], 2)

[
    f"[{round(HFEV1.bins[i], 2)}; {last_elem if i == (len(HFEV1.bins)-1) else round(HFEV1.bins[i+1], 2)})"
    for i in range(len(HFEV1.bins))
]

['[2.0; 2.1)',
 '[2.1; 2.2)',
 '[2.2; 2.3)',
 '[2.3; 2.4)',
 '[2.4; 2.5)',
 '[2.5; 2.6)',
 '[2.6; 2.7)',
 '[2.7; 2.8)',
 '[2.8; 2.9)',
 '[2.9; 3.0)',
 '[3.0; 3.1)',
 '[3.1; 3.2)',
 '[3.2; 3.3)',
 '[3.3; 3.4)',
 '[3.4; 3.5)',
 '[3.5; 3.6)',
 '[3.6; 3.7)',
 '[3.7; 3.8)',
 '[3.8; 3.9)',
 '[3.9; 4.0)',
 '[4.0; 4.1)',
 '[4.1; 4.2)',
 '[4.2; 4.3)',
 '[4.3; 4.4)',
 '[4.4; 4.5)',
 '[4.5; 4.6)',
 '[4.6; 4.7)',
 '[4.7; 4.8)',
 '[4.8; 4.9)',
 '[4.9; 5.0)',
 '[5.0; 5.1)',
 '[5.1; 5.2)',
 '[5.2; 5.3)',
 '[5.3; 5.4)',
 '[5.4; 5.5)',
 '[5.5; 5.6)',
 '[5.6; 5.7)',
 '[5.7; 5.8)',
 '[5.8; 5.9)',
 '[5.9; 6)']